In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
lr = 3e-3
eval_iters = 250

In [6]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [7]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [8]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    xi = torch.randint(len(data) - block_size, (batch_size,)) # random starting indices
    # print(f"Training Integers: {xi}")
    # input batch
    x = torch.stack([data[i:i+block_size] for i in xi])
    # target batch    
    y = torch.stack([data[i+1:i+1+block_size] for i in xi])
    x, y = x.to(device), y.to(device) # send to cuda -> parallelize
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # type: ignore
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # type: ignore
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # type: ignore
    return out

In [ ]:
# GPT language model
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B=batch size, T=time, C=channels (vocab size)
            
            # reshape logits and targets for cross-entropy loss
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)  # get predictions
            logits = logits[:, -1, :] # (B, C) last time step
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel(vocab_size).to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)        

In [ ]:
#PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # type: ignore
    optimizer.step()
print(loss.item()) # type: ignore